# xarray fussing

In [85]:
# imports
from importlib import reload
import os
import glob

import numpy as np
import xarray

from mhw_analysis import utils as mhw_utils

# Load up an SST

In [4]:
noaa_path = os.getenv("NOAA_OI")
all_sst_files = glob.glob(os.path.join(noaa_path, 'sst*nc'))
ifile = all_sst_files[0]

In [34]:
ds = xarray.open_dataset(ifile)
#_ = ds.sst.data
ds

<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 366)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01 2008-01-02 ... 2008-12-31
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.375 359.625 359.875
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    Conventions:    CF-1.5
    title:          NOAA High-resolution Blended Analysis: Daily Values using...
    institution:    NOAA/NCDC
    source:         NOAA/NCDC  ftp://eclipse.ncdc.noaa.gov/pub/OI-daily-v2/
    comment:        Reynolds, et al., 2007: Daily High-Resolution-Blended Ana...
    history:        Thu Aug 24 13:48:53 2017: ncatted -O -a References,global...
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....

# Grab a region

In [29]:
ds.sst.sel(lon=slice(340.,350.), lat=slice(45.,60.), time=slice("2008-01-1","2008-01-01"))

<xarray.DataArray 'sst' (time: 1, lat: 60, lon: 40)>
array([[[14.99    , 14.969999, ..., 14.15    , 14.12    ],
        [14.9     , 14.86    , ..., 14.009999, 13.99    ],
        ...,
        [ 9.86    ,  9.969999, ...,  9.55    ,  9.48    ],
        [ 9.849999,  9.889999, ...,  9.49    ,  9.46    ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01
  * lat      (lat) float32 45.125 45.375 45.625 45.875 ... 59.375 59.625 59.875
  * lon      (lon) float32 340.125 340.375 340.625 ... 349.375 349.625 349.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

# Grab around lon=0

## Roll the DataArray -- Too slow

In [61]:
sst = ds.sst
_  = sst.data

In [62]:
roll_sst = sst.roll(shifts={'lon':1440//2}, roll_coords=['lon'])
roll_sst

<xarray.DataArray 'sst' (time: 366, lat: 720, lon: 1440)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.6899999, -1.6899999, -1.6899999, ..., -1.6899999,
         -1.6899999, -1.6899999],
        [-1.6899999, -1.6899999, -1.6899999, ..., -1.6899999,
         -1.6899999, -1.6899999],
        [-1.6899999, -1.6899999, -1.6899999, ..., -1.6899999,
         -1.6899999, -1.6899999]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [-1.74     , -1.74     , -1.74     , ..., -1.74     ,
         -1.74     , -1.74     ],
        [-1.74     , -1.74     , -1.74     , ..., -1.74     ,
         -1.74     , -1.74     ],
        [-1.74     , -1.74     , -1.74     , ..., -1.74     ,
         -1.74     , -1.74     ]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.73     , -1.73     , -1.73     , ..., -1.73     ,
         -1.73     , -1.73     ],
        [-1.73     , -1.73     , -1.73     , ..., -1.73     ,
         -1.73     , -1.73     ],
        [-1.73     , -1.73     , -1.73     , ..., -1.73     ,
         -1.73     , -1.73     ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01 2008-01-02 ... 2008-12-31
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 180.125 180.375 180.625 ... 179.375 179.625 179.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

## Combine 2 regions

In [58]:
lon = [350., 10.]

In [83]:
rangeA = ds.sst.sel(lon=slice(lon[0], 360.))#.assign_coords(lon=(((rangeA.lon + 180) % 360) - 180))
rangeA

<xarray.DataArray 'sst' (time: 366, lat: 720, lon: 40)>
array([[[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.69, -1.69, ..., -1.69, -1.69],
        [-1.69, -1.69, ..., -1.69, -1.69]],

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.68, -1.68, ..., -1.68, -1.68],
        [-1.68, -1.68, ..., -1.68, -1.68]],

       ...,

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.74, -1.74, ..., -1.74, -1.74],
        [-1.74, -1.74, ..., -1.74, -1.74]],

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.73, -1.73, ..., -1.73, -1.73],
        [-1.73, -1.73, ..., -1.73, -1.73]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01 2008-01-02 ... 2008-12-31
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 350.125 350.375 350.625 ... 359.375 359.625 359.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

In [75]:
rangeA.lon

<xarray.DataArray 'lon' (lon: 40)>
array([-9.875, -9.625, -9.375, -9.125, -8.875, -8.625, -8.375, -8.125, -7.875,
       -7.625, -7.375, -7.125, -6.875, -6.625, -6.375, -6.125, -5.875, -5.625,
       -5.375, -5.125, -4.875, -4.625, -4.375, -4.125, -3.875, -3.625, -3.375,
       -3.125, -2.875, -2.625, -2.375, -2.125, -1.875, -1.625, -1.375, -1.125,
       -0.875, -0.625, -0.375, -0.125], dtype=float32)
Coordinates:
  * lon      (lon) float32 -9.875 -9.625 -9.375 -9.125 ... -0.625 -0.375 -0.125

In [76]:
rangeB = ds.sst.sel(lon=slice(0., lon[1]))
rangeB

<xarray.DataArray 'sst' (time: 366, lat: 720, lon: 40)>
array([[[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.69, -1.69, ..., -1.69, -1.69],
        [-1.69, -1.69, ..., -1.69, -1.69]],

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.68, -1.68, ..., -1.68, -1.68],
        [-1.68, -1.68, ..., -1.68, -1.68]],

       ...,

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.74, -1.74, ..., -1.73, -1.73],
        [-1.74, -1.74, ..., -1.74, -1.74]],

       [[  nan,   nan, ...,   nan,   nan],
        [  nan,   nan, ...,   nan,   nan],
        ...,
        [-1.73, -1.73, ..., -1.73, -1.73],
        [-1.73, -1.73, ..., -1.73, -1.73]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01 2008-01-02 ... 2008-12-31
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 9.125 9.375 9.625 9.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

## Concatenate

In [84]:
frange = xarray.concat([rangeA, rangeB], dim='lon')
frange

<xarray.DataArray 'sst' (time: 366, lat: 720, lon: 80)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.6899999, -1.6899999, -1.6899999, ..., -1.68     ,
         -1.68     , -1.68     ],
        [-1.6899999, -1.6899999, -1.6899999, ..., -1.6899999,
         -1.6899999, -1.6899999],
        [-1.6899999, -1.6899999, -1.6899999, ..., -1.6899999,
         -1.6899999, -1.6899999]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [-1.73     , -1.73     , -1.73     , ..., -1.73     ,
         -1.73     , -1.73     ],
        [-1.74     , -1.74     , -1.74     , ..., -1.73     ,
         -1.73     , -1.73     ],
        [-1.74     , -1.74     , -1.74     , ..., -1.74     ,
         -1.74     , -1.74     ]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.73     , -1.73     , -1.73     , ..., -1.7199999,
         -1.7199999, -1.7199999],
        [-1.73     , -1.73     , -1.73     , ..., -1.73     ,
         -1.73     , -1.73     ],
        [-1.73     , -1.73     , -1.73     , ..., -1.73     ,
         -1.73     , -1.73     ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * time     (time) datetime64[ns] 2008-01-01 2008-01-02 ... 2008-12-31
  * lon      (lon) float64 350.1 350.4 350.6 350.9 ... 9.125 9.375 9.625 9.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

In [79]:
frange.lon

<xarray.DataArray 'lon' (lon: 80)>
array([-9.875, -9.625, -9.375, -9.125, -8.875, -8.625, -8.375, -8.125, -7.875,
       -7.625, -7.375, -7.125, -6.875, -6.625, -6.375, -6.125, -5.875, -5.625,
       -5.375, -5.125, -4.875, -4.625, -4.375, -4.125, -3.875, -3.625, -3.375,
       -3.125, -2.875, -2.625, -2.375, -2.125, -1.875, -1.625, -1.375, -1.125,
       -0.875, -0.625, -0.375, -0.125,  0.125,  0.375,  0.625,  0.875,  1.125,
        1.375,  1.625,  1.875,  2.125,  2.375,  2.625,  2.875,  3.125,  3.375,
        3.625,  3.875,  4.125,  4.375,  4.625,  4.875,  5.125,  5.375,  5.625,
        5.875,  6.125,  6.375,  6.625,  6.875,  7.125,  7.375,  7.625,  7.875,
        8.125,  8.375,  8.625,  8.875,  9.125,  9.375,  9.625,  9.875])
Coordinates:
  * lon      (lon) float64 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875

In [48]:
tst = xarray.DataArray(np.zeros((10,10)), coords=[tstc, tstc2])

In [49]:
tst

<xarray.DataArray (st: 10, st2: 10)>
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Coordinates:
  * st       (st) int64 5 6 7 8 9 0 1 2 3 4
  * st2      (st2) int64 0 1 2 3 4 5 6 7 8 9

In [80]:
rangeC = ds.sst.sel(lon=slice(0.125, 0.375))
rangeC

<xarray.DataArray 'sst' (time: 366, lat: 720, lon: 2)>
array([[[  nan,   nan],
        [  nan,   nan],
        ...,
        [-1.69, -1.69],
        [-1.69, -1.69]],

       [[  nan,   nan],
        [  nan,   nan],
        ...,
        [-1.68, -1.68],
        [-1.68, -1.68]],

       ...,

       [[  nan,   nan],
        [  nan,   nan],
        ...,
        [-1.74, -1.74],
        [-1.74, -1.74]],

       [[  nan,   nan],
        [  nan,   nan],
        ...,
        [-1.73, -1.73],
        [-1.73, -1.73]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01 2008-01-02 ... 2008-12-31
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

# Test

In [88]:
ds = xarray.open_dataset(ifile)
da = ds.sst.sel(time='2008-01-01')
da

<xarray.DataArray 'sst' (lat: 720, lon: 1440)>
[1036800 values with dtype=float32]
Coordinates:
    time     datetime64[ns] 2008-01-01
  * lat      (lat) float32 -89.875 -89.625 -89.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.375 359.625 359.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

In [89]:
_ = da.data

## Sub image 1

In [92]:
reload(mhw_utils)

<module 'mhw_analysis.utils' from '/home/xavier/Projects/Oceanography/Python/mhw_analysis/mhw_analysis/utils.py'>

In [90]:
sub1 = mhw_utils.grab_geo_subimg(da, [-10., 10.], [10., 60.])
sub1

<xarray.DataArray 'sst' (lat: 80, lon: 200)>
array([[26.619999, 26.49    , 26.38    , ..., 28.32    , 28.289999, 28.31    ],
       [26.789999, 26.699999, 26.599998, ..., 28.189999, 28.109999, 28.109999],
       [26.82    , 26.789999, 26.75    , ..., 28.06    , 27.96    , 28.      ],
       ...,
       [      nan,       nan,       nan, ..., 27.099998, 27.14    , 27.199999],
       [      nan,       nan,       nan, ..., 26.939999, 27.029999, 27.119999],
       [      nan,       nan,       nan, ..., 26.65    , 26.75    , 26.89    ]],
      dtype=float32)
Coordinates:
    time     datetime64[ns] 2008-01-01
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
  * lon      (lon) float32 10.125 10.375 10.625 10.875 ... 59.375 59.625 59.875
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

## Sub image 2

In [95]:
sub2 = mhw_utils.grab_geo_subimg(da, [-10., 10.], [340., 30.])#, fix_coord=True)
sub2

<xarray.DataArray 'sst' (lat: 80, lon: 200)>
array([[25.599998, 25.529999, 25.49    , ...,       nan,       nan,
              nan],
       [25.619999, 25.59    , 25.55    , ...,       nan,       nan,
              nan],
       [25.66    , 25.64    , 25.65    , ...,       nan,       nan,
              nan],
       ...,
       [26.75    , 26.859999, 27.07    , ...,       nan,       nan,
              nan],
       [26.89    , 27.01    , 27.189999, ...,       nan,       nan,
              nan],
       [27.05    , 27.21    , 27.359999, ...,       nan,       nan,
              nan]], dtype=float32)
Coordinates:
    time     datetime64[ns] 2008-01-01
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
  * lon      (lon) float64 340.1 340.4 340.6 340.9 ... 29.12 29.38 29.62 29.88
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  35.95]

In [96]:
sub2.lon

<xarray.DataArray 'lon' (lon: 200)>
array([3.40125e+02, 3.40375e+02, 3.40625e+02, 3.40875e+02, 3.41125e+02,
       3.41375e+02, 3.41625e+02, 3.41875e+02, 3.42125e+02, 3.42375e+02,
       3.42625e+02, 3.42875e+02, 3.43125e+02, 3.43375e+02, 3.43625e+02,
       3.43875e+02, 3.44125e+02, 3.44375e+02, 3.44625e+02, 3.44875e+02,
       3.45125e+02, 3.45375e+02, 3.45625e+02, 3.45875e+02, 3.46125e+02,
       3.46375e+02, 3.46625e+02, 3.46875e+02, 3.47125e+02, 3.47375e+02,
       3.47625e+02, 3.47875e+02, 3.48125e+02, 3.48375e+02, 3.48625e+02,
       3.48875e+02, 3.49125e+02, 3.49375e+02, 3.49625e+02, 3.49875e+02,
       3.50125e+02, 3.50375e+02, 3.50625e+02, 3.50875e+02, 3.51125e+02,
       3.51375e+02, 3.51625e+02, 3.51875e+02, 3.52125e+02, 3.52375e+02,
       3.52625e+02, 3.52875e+02, 3.53125e+02, 3.53375e+02, 3.53625e+02,
       3.53875e+02, 3.54125e+02, 3.54375e+02, 3.54625e+02, 3.54875e+02,
       3.55125e+02, 3.55375e+02, 3.55625e+02, 3.55875e+02, 3.56125e+02,
       3.56375e+02, 3.56625e+02, 3.56875e+02, 3.57125e+02, 3.57375e+02,
       3.57625e+02, 3.57875e+02, 3.58125e+02, 3.58375e+02, 3.58625e+02,
       3.58875e+02, 3.59125e+02, 3.59375e+02, 3.59625e+02, 3.59875e+02,
       1.25000e-01, 3.75000e-01, 6.25000e-01, 8.75000e-01, 1.12500e+00,
       1.37500e+00, 1.62500e+00, 1.87500e+00, 2.12500e+00, 2.37500e+00,
       2.62500e+00, 2.87500e+00, 3.12500e+00, 3.37500e+00, 3.62500e+00,
       3.87500e+00, 4.12500e+00, 4.37500e+00, 4.62500e+00, 4.87500e+00,
       5.12500e+00, 5.37500e+00, 5.62500e+00, 5.87500e+00, 6.12500e+00,
       6.37500e+00, 6.62500e+00, 6.87500e+00, 7.12500e+00, 7.37500e+00,
       7.62500e+00, 7.87500e+00, 8.12500e+00, 8.37500e+00, 8.62500e+00,
       8.87500e+00, 9.12500e+00, 9.37500e+00, 9.62500e+00, 9.87500e+00,
       1.01250e+01, 1.03750e+01, 1.06250e+01, 1.08750e+01, 1.11250e+01,
       1.13750e+01, 1.16250e+01, 1.18750e+01, 1.21250e+01, 1.23750e+01,
       1.26250e+01, 1.28750e+01, 1.31250e+01, 1.33750e+01, 1.36250e+01,
       1.38750e+01, 1.41250e+01, 1.43750e+01, 1.46250e+01, 1.48750e+01,
       1.51250e+01, 1.53750e+01, 1.56250e+01, 1.58750e+01, 1.61250e+01,
       1.63750e+01, 1.66250e+01, 1.68750e+01, 1.71250e+01, 1.73750e+01,
       1.76250e+01, 1.78750e+01, 1.81250e+01, 1.83750e+01, 1.86250e+01,
       1.88750e+01, 1.91250e+01, 1.93750e+01, 1.96250e+01, 1.98750e+01,
       2.01250e+01, 2.03750e+01, 2.06250e+01, 2.08750e+01, 2.11250e+01,
       2.13750e+01, 2.16250e+01, 2.18750e+01, 2.21250e+01, 2.23750e+01,
       2.26250e+01, 2.28750e+01, 2.31250e+01, 2.33750e+01, 2.36250e+01,
       2.38750e+01, 2.41250e+01, 2.43750e+01, 2.46250e+01, 2.48750e+01,
       2.51250e+01, 2.53750e+01, 2.56250e+01, 2.58750e+01, 2.61250e+01,
       2.63750e+01, 2.66250e+01, 2.68750e+01, 2.71250e+01, 2.73750e+01,
       2.76250e+01, 2.78750e+01, 2.81250e+01, 2.83750e+01, 2.86250e+01,
       2.88750e+01, 2.91250e+01, 2.93750e+01, 2.96250e+01, 2.98750e+01])
Coordinates:
    time     datetime64[ns] 2008-01-01
  * lon      (lon) float64 340.1 340.4 340.6 340.9 ... 29.12 29.38 29.62 29.88
Attributes:
    long_name:      Longitude
    standard_name:  longitude
    units:          degrees_east
    actual_range:   [1.25000e-01 3.59875e+02]
    axis:           X